<a href="https://colab.research.google.com/github/srujith7/AD-075/blob/main/textclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
dataset=pd.read_csv("/content/hate_speech.csv")
dataset.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [2]:
dataset.shape

(5242, 3)

In [3]:
dataset.label.value_counts()

,count
label,
0,3000
1,2242


In [4]:
for index,tweet in enumerate(dataset["tweet"][10:15]):
  print(index+1,"-",tweet)

1 -  â #ireland consumer price index (mom) climbed from previous 0.2% to 0.5% in may   #blog #silver #gold #forex
2 - we are so selfish. #orlando #standwithorlando #pulseshooting #orlandoshooting #biggerproblems #selfish #heabreaking   #values #love #
3 - i get to see my daddy today!!   #80days #gettingfed
4 - ouch...junior is angryð#got7 #junior #yugyoem   #omg 
5 - i am thankful for having a paner. #thankful #positive     


In [5]:
import re
def clean_text(text):
  text = re.sub(r'[^a-zA-Z\']', ' ', text)
  text = re.sub(r'[^\x00-\x7F]+', '', text)
  text =  text.lower()
  return text

In [6]:
dataset['clean_text'] = dataset.tweet.apply(lambda x: clean_text(x))

In [7]:
dataset.head(3)

,id,label,tweet,clean_text
0,1,0,@user when a father is dysfunctional and is s...,user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...,user user thanks for lyft credit i can't us...
2,3,0,bihday your majesty,bihday your majesty


In [8]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop_words=stopwords.words('english')
len(stop_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


179

In [9]:
def gen_freq(text):
  word_list=[]
  for word in text.split():
    word_list.extend(word)
  word_freq=pd.Series(word_list).value_counts()
  word_freq=word_freq.drop(stop_words,errors='ignore')
  return word_freq

In [10]:
def any_neg(words):
  for word in words:
    if word in ['no','nor','not'] or re.search:
      return 1
  else:
    return 0

In [12]:
import pandas as pd
import re
from nltk.corpus import stopwords
import nltk

# Download stopwords if you haven't already
nltk.download('stopwords')
stop_words = stopwords.words('english')

# ... (Your previous code to load and clean the dataset) ...

# Create new features
dataset['word_count'] = dataset['clean_text'].apply(lambda x: len(str(x).split()))
dataset['char_count'] = dataset['clean_text'].apply(lambda x: len(str(x)))
dataset['is_question'] = dataset['clean_text'].apply(lambda x: 1 if '?' in str(x) else 0)

# Function to identify rare words (You'll need to define a threshold for rarity)
def find_rare_words(text, threshold=5):
    word_freq = pd.Series(text.split()).value_counts()
    rare_words = word_freq[word_freq < threshold].index.tolist()
    return rare_words

# Function to check for rare words in a text
def has_rare_words(text):
    rare_words_list = find_rare_words(text)  # Using the find_rare_words function
    for word in text.split():
        if word in rare_words_list:
            return 1
    return 0

dataset['any_rare'] = dataset['clean_text'].apply(lambda x: has_rare_words(x))


def any_neg(text):
    for word in text.split():
        if word in ['no', 'nor', 'not']:
            return 1
        # Remove the re.search as it was incomplete and causing issues
    return 0
dataset['any_neg'] = dataset['clean_text'].apply(lambda x: any_neg(x))


# Now you can split the data
from sklearn.model_selection import train_test_split
x = dataset[['word_count', 'any_neg', 'any_rare', 'char_count', 'is_question']]
y = dataset.label
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
from sklearn.model_selection import train_test_split
x = dataset[['word_count', 'any_neg', 'any_rare', 'char_count', 'is_question']]
y = dataset.label
x_train, x_test, y_train,  y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [19]:
from sklearn.metrics import accuracy_score
print("Accuracy: ",accuracy_score(y_test,pred)*100,"%")

Accuracy:  57.578646329837945 %


In [17]:
from sklearn.naive_bayes import GaussianNB
model=GaussianNB()
model=model.fit(x_train,y_train)
pred=model.predict(x_test)


In [23]:
model.predict(x_test[5:10])

array([0, 0, 0, 0, 0])

In [24]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier()
model.fit(x_train,y_train)
model_pred=model.predict(x_test).astype(int)

In [25]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
print(accuracy_score(y_test,model_pred))
print(confusion_matrix(y_test,model_pred))
print(classification_report(y_test,model_pred))

0.5881792183031459
[[412 187]
 [245 205]]
              precision    recall  f1-score   support

           0       0.63      0.69      0.66       599
           1       0.52      0.46      0.49       450

    accuracy                           0.59      1049
   macro avg       0.58      0.57      0.57      1049
weighted avg       0.58      0.59      0.58      1049



In [26]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
print("Accuracy:",accuracy_score(y_test,model_pred)*100,"%")
print("Confusion Matrxix:\n",confusion_matrix(y_test,model_pred))

print("Classification Report:\n",classification_report(y_test,model_pred))

Accuracy: 58.81792183031459 %
Confusion Matrxix:
 [[412 187]
 [245 205]]
Classification Report:
               precision    recall  f1-score   support

           0       0.63      0.69      0.66       599
           1       0.52      0.46      0.49       450

    accuracy                           0.59      1049
   macro avg       0.58      0.57      0.57      1049
weighted avg       0.58      0.59      0.58      1049

